In [ ]:
import findspark
findspark.init("/usr/local/spark")

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("ML").getOrCreate()

In [ ]:
df = spark.read.csv("churn.csv",inferSchema=True, header=True)

In [ ]:
#df.toPandas

In [ ]:
# Explore, clean, analyse & visualise the data

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
cat_id = StringIndexer(inputCol= "Geography", outputCol='Geo').fit(df)

In [ ]:
df = cat_id.transform(df)

In [ ]:
df.select('Geo','Geography').show(5)

In [ ]:
cat_id1 = StringIndexer(inputCol= "Gender", outputCol='Gender1').fit(df)

In [ ]:
df = cat_id1.transform(df)

In [ ]:
df.select('Gender','Gender1').show(5)

In [ ]:
cat_one = OneHotEncoder(inputCol="Geo", outputCol="Geo_encoded")

In [ ]:
df=cat_one.transform(df)

In [ ]:
df.select('Geo','Geo_encoded').show()

In [ ]:
cat_two = OneHotEncoder(inputCol="Gender1", outputCol="Gen_encoded")

In [ ]:
df=cat_two.transform(df)

In [ ]:
df.columns

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
df_assembler = VectorAssembler(inputCols=['CreditScore','Age','Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Geo_encoded',
 'Gen_encoded'], outputCol="features")

In [ ]:
df = df_assembler.transform(df)

In [ ]:
df.select('features').show(5)

In [ ]:
final_df=df.select("features","Exited")

In [ ]:
final_df.show(5)

In [ ]:
train_data,test_data = final_df.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier

In [ ]:
lr = LogisticRegression(labelCol='Exited')

In [ ]:
model = lr.fit(train_data)

In [ ]:
pred_churn=model.evaluate(test_data)

In [ ]:
pred_churn.predictions.show(15)

In [ ]:
type(pred_churn)

In [ ]:
pred_churn.predictions.collect()

In [ ]:
pred = model.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [ ]:
eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Exited')

In [ ]:
eval.evaluate(pred)

In [ ]:
help(BinaryClassificationEvaluator)

In [ ]:
dtree = DecisionTreeClassifier(featuresCol='features', labelCol='Exited')

In [ ]:
model_tree = dtree.fit(train_data)

In [ ]:
pred_tree=model_tree.transform(test_data)

In [ ]:
pred_tree

In [ ]:
pred_tree.show()

In [ ]:
eval_tree = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Exited')

In [ ]:
acc = eval_tree.evaluate(pred_tree)

In [ ]:
acc

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_origin = pred_tree.select('Exited').collect()

In [ ]:
y_predicted = pred_tree.select('prediction').collect()

In [ ]:
cm = confusion_matrix(y_true=y_origin,y_pred=y_predicted)

In [ ]:
cm

In [ ]:
197/(197+86)

In [ ]:
197/(201+197)